In [70]:
import pandas as pd
import numpy as np
import re
import inflect
import pandas as pd
from neo4j import GraphDatabase
import yaml
import shutil
import os
from pathlib import Path



In [71]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [72]:

p = inflect.engine()

In [73]:
p.singular_noun('alliums')

'allium'

In [74]:
sheet_id = PARAM["google_sheet_id"]
sheet_name = PARAM["google_sheet_node_name"]
taxonomy_name = PARAM["google_sheet_taxonomy_name"]
synonym_name = PARAM["google_sheet_synonyms"]
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(url)

taxonomy_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={taxonomy_name}"
taxonomy = pd.read_csv(taxonomy_url)

synonym_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={synonym_name}"
synonyms = pd.read_csv(synonym_url)

In [75]:
taxonomy_sons_index = taxonomy.groupby("from").groups
taxonomy_sons = {}
for parent in taxonomy_sons_index:
    taxonomy_sons[parent] = list(taxonomy.iloc[taxonomy_sons_index[parent]]["to"].values)

In [76]:
print (taxonomy_sons)

{'allium': ['onion', 'garlic', 'leek', 'chive'], 'cucurbit': ['squash', 'pumpkin', 'zucchini', 'gourd', 'cucumber', 'melon'], 'fruit tree': ['apple', 'apricot', 'pear'], 'legume': ['alfalfa', 'lentil', 'lupin', 'pea', 'peanut', 'soybean', 'clover', 'bush bean', 'pole bean', 'fava bean', 'vetch', 'crownvetch'], 'nightshade': ['tomato', 'potato', 'eggplant', 'chili pepper', 'pepper']}


In [77]:
#df = pd.read_csv("companion_plant_wikipedia - after_manual.tsv", sep="\t")
df.head()

,Common name,Type,Scientific name,Helps,Helped by,Attracts,-Repels/+distracts,Avoid,Comments
0,african spider plant,vegetable,Cleome gynandra,NaN,NaN,NaN,NaN,NaN,NaN
1,alfalfa,other,Medicago sativa,cotton,NaN,"assassin bug, big-eyed bug, ladybug, parasitic...",lygus bugs,"tomatoes, fava beans","used by farmers to reduce cotton pests, a good..."
2,allium,vegetable,Allium,"fruit trees, tomatoes, peppers, potatoes, bras...","carrots, tomatoes, carrots, african spider pla...",thrips,"rabbits, slugs, aphids, carrot fly, cabbage lo...","beans, peas",alliums are a family of plants which include o...
3,alyssum,flower,Lobularia maritima,"grapes, lettuce",NaN,"predatory insects, minute pirate bugs, parasit...",aphids,NaN,because they attract syrphidae they help reduc...
4,Angelica archangelica,vegetable,Angelica archangelica,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
synonymes = synonyms.set_index("name")["synonym"].to_dict()

In [79]:
nodes_plant = set()
nodes_animal = set()

nodes_info = {}

plant_plant_help = set()
plant_plant_avoid = set()
node_animal_attract = set()
node_animal_repel = set()

In [80]:
wrong_singular = ["asparagus", "gladiolus", "kentucky bluegrass", "Dianthus", "lemon grass", "rye-grass", "sudan grass", "Cosmos", "cress"]

In [81]:
def normalize(words):
    for s in synonymes:
        if s in words:
            words.remove(s)
            words.append(synonymes[s])
    
    if "" in words:
        words.remove("")

    words = [p.singular_noun(x) if p.singular_noun(x) != False and x not in wrong_singular else x for x in words]

    words = [re.sub(r'\s+', ' ', x) for x in words]

    final_word_list = []
    for x in words:
        if x in taxonomy_sons:
            final_word_list += taxonomy_sons[x]
        else:
            final_word_list.append(x)

    return final_word_list

In [82]:
normalize(["legume", "tomato"])

['alfalfa',
 'lentil',
 'lupin',
 'pea',
 'peanut',
 'soybean',
 'clover',
 'bush bean',
 'pole bean',
 'fava bean',
 'vetch',
 'crownvetch',
 'tomato']

In [83]:


for index, row in df.iterrows():
    common_names = [row["Common name"]]
    common_names = normalize(common_names)
    

    nodes_plant.update(common_names)


    helps = [x.strip() for x in str(row["Helps"]).split(",") if not pd.isna(row["Helps"]) ]
    helps = normalize(helps)
    nodes_plant.update(helps)

    for h in helps:
        for common_name in common_names:
            if h != common_name:
                plant_plant_help.add((common_name, h))

    helped_by = [x.strip() for x in str(row["Helped by"]).split(",") if not pd.isna(row["Helped by"]) ]
    helped_by = normalize(helped_by)
    nodes_plant.update(helped_by)

    for h in helped_by:
        for common_name in common_names:
            if h != common_name:
                plant_plant_help.add((h, common_name))

    attracts = [x.strip() for x in str(row["Attracts"]).split(",") if not pd.isna(row["Attracts"]) ]
    attracts = normalize(attracts)
    nodes_animal.update(attracts)

    for a in attracts:
        for common_name in common_names:
            node_animal_attract.add((common_name, a))

    repel = [x.strip() for x in str(row["-Repels/+distracts"]).split(",") if not pd.isna(row["-Repels/+distracts"]) ]
    repel = normalize(repel)
    nodes_animal.update(repel)

    for r in repel:
        for common_name in common_names:
            node_animal_repel.add((common_name, r))

    avoid = [x.strip() for x in str(row["Avoid"]).split(",") if not pd.isna(row["Avoid"]) ]
    avoid = normalize(avoid)
    nodes_plant.update(avoid)

    for a in avoid:
        for common_name in common_names:
            plant_plant_avoid.add(tuple(sorted(list([common_name, a]))))

    sci_name = ""
    if not pd.isna(row["Scientific name"]):
        sci_name = row["Scientific name"]

    type = ""
    if not pd.isna(row["Type"]):
        type = row["Type"]

    comment = ""
    if not pd.isna(row["Comments"]):
        comment = re.sub(r'\s+', ' ', row["Comments"])

    for common_name in common_names:
        if common_name not in nodes_info:
            nodes_info[common_name] = {"scientific name": sci_name, "type": type, "comment": comment}
        else:
            if len(common_names) == 1:
                nodes_info[common_name] = {"scientific name": sci_name, "type": type, "comment": comment}

In [84]:
if "almost everything" in nodes_plant:
    nodes_plant.remove("almost everything")

In [85]:


content = "name\tscientific_name\ttype\tcomment\n"
for n in nodes_plant:
    if n in nodes_info:
        content += f"{n}\t{nodes_info[n]['scientific name']}\t{nodes_info[n]['type']}\t{nodes_info[n]['comment']}\n"
    else:
        content += f"{n}\t""\t""\t""\n"

output_file = open("data/nodes_plant.tsv", 'w')
output_file.write(content)
output_file.close()


In [86]:
content = "name\n"
for n in nodes_animal:
    content += f"{n}\n"

output_file = open("data/nodes_animal.tsv", 'w')
output_file.write(content)
output_file.close()

In [87]:
content = "from\tto\n"
for (from_, to_) in plant_plant_help:
    content += f"{from_}\t{to_}\n"

output_file = open("data/plant_plant_help.tsv", 'w')
output_file.write(content)
output_file.close()

In [88]:
content = "from\tto\n"
for (from_, to_) in node_animal_attract:
    content += f"{from_}\t{to_}\n"

output_file = open("data/node_animal_attract.tsv", 'w')
output_file.write(content)
output_file.close()

In [89]:
content = "from\tto\n"
for (from_, to_) in node_animal_repel:
    content += f"{from_}\t{to_}\n"

output_file = open("data/node_animal_repel.tsv", 'w')
output_file.write(content)
output_file.close()

In [90]:
content = "from\tto\n"
for (from_, to_) in plant_plant_avoid:
    content += f"{from_}\t{to_}\n"

output_file = open("data/plant_plant_avoid.tsv", 'w')
output_file.write(content)
output_file.close()

In [91]:
taxonomy.to_csv("data/taxonomy.tsv", sep="\t", index=False)

In [92]:
url = "bolt://localhost:7687"
driver = GraphDatabase.driver(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))

In [93]:
with driver.session() as session:
    # Drop constraints / indices
    for constraint in session.run("CALL db.constraints"):

        session.run("DROP CONSTRAINT " + constraint[0] + ";")

    # delete all nodes    
    session.run("MATCH (n) DETACH DELETE n")

In [94]:
src = "data"

home = str(Path.home())

dst = os.path.join(home, ".Neo4jDesktop", "relate-data", "dbmss", PARAM["neo4j_project_id"], "import")

for src_dir, dirs, files in os.walk(src):
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst, file_)
        shutil.copy(src_file, dst_file)


In [95]:
for line in open("neo4j_command.txt", 'r'):
    if line.strip() != "":
        with driver.session() as session:
            session.run(line.strip())